In [64]:
import pandas as pd
import numpy as np

In [65]:
#Read relevant tables
#Hogar: Description of the households
#Gasto: Detailed description of spending of each household
hogar=pd.read_csv('EPFhogar_2021.csv', sep='\t')
gasto=pd.read_csv('EPFgastos_2021.csv', sep='\t')

In [66]:
#Keep repelvant columns
hogar=hogar[['NUMERO','CCAA','NMIEM5','IMPEXAC']]

In [67]:
#Make income annual
hogar.IMPEXAC=hogar.IMPEXAC*12

In [68]:
#Make the number of underage into a binary variable of wether there's underage
hogar.NMIEM5=np.sign(hogar.NMIEM5)

In [69]:
hogar.NMIEM5.value_counts()

0    13640
1     5754
Name: NMIEM5, dtype: int64

In [70]:
#The spent values are in inflated version (multiplied by the demographic weights). Divide again to get to real family spending
gasto['spending']=gasto['GASTO']/gasto['FACTOR']

In [71]:
#Get the 2--digit product category (COICOP)
gasto['COICOP']=gasto.CODIGO.astype(str).str[:2]

In [72]:
#Make 4 spending categories by grouping COICOPs
gasto['Category']=np.where(gasto.COICOP.isin(['01','02','11']),'food',
                          np.where(gasto.COICOP.isin(['04','05']),'housing',
                                  np.where(gasto.COICOP.isin(['07']), 'transport','other')))

In [73]:
#Pivot for spending in 4 categories for each family
aggregate=pd.pivot_table(gasto, values='spending', index=['NUMERO'],
                    columns=['Category'], aggfunc=np.sum)

In [74]:
aggregate=aggregate.fillna(0)

In [75]:
hogar=hogar.merge(aggregate.reset_index(),on='NUMERO', how='left')

In [76]:
hogar.columns=['hogar_id','ccaa','menor','ingreso','alimento','vivienda','otro','transporte']

In [78]:
hogar.to_csv('SAS_hogares_2021.csv',index=False)

In [79]:
hogar

,hogar_id,ccaa,menor,ingreso,alimento,vivienda,otro,transporte
0,1,8,0,9384,1511.768669,4436.358398,355.999998,0.000000
1,2,3,0,15396,4862.049060,8622.876213,2133.019522,0.000000
2,3,5,0,22320,7545.543648,7341.991905,4284.857830,3711.480046
3,4,11,0,16200,2683.471194,4314.946813,4599.522104,0.000000
4,5,10,0,14472,8037.463035,7966.047788,2337.196691,0.000000
...,...,...,...,...,...,...,...,...
19389,19390,11,1,9936,3276.342246,3229.199104,3624.898126,3499.652799
19390,19391,8,0,14256,3810.710503,4193.611686,1178.382301,0.000000
19391,19392,9,0,21132,3654.975616,18317.921157,2962.964186,1921.639222
19392,19393,8,1,40440,12145.067082,8843.762109,14029.869121,30445.323783
